 # 4.2 Volatility Spillover Measures Using Generalized Variance Decomposition (GVD)



 Diebold and Yilmaz (2009, 2012) propose measuring volatility spillovers using the forecast error variance decomposition from a vector autoregressive (VAR) model.



 The VAR($p$) model is specified as:



 $$

 Y_t = \sum_{i=1}^p \Phi_i Y_{t-i} + \varepsilon_t,

 $$



 where $Y_t$ is an $N \times 1$ vector of realized variances, $\Phi_i$ are coefficient matrices, and $\varepsilon_t$ is a white noise error vector with covariance matrix $\Sigma$.



 The moving average representation is:



 $$

 Y_t = \sum_{i=0}^\infty A_i \varepsilon_{t-i}

 $$



 where matrices $A_i$ satisfy the recursion $A_i = \sum_{j=1}^p \Phi_j A_{i-j}$, with $A_0 = I_N$.



The forecast error variance decomposition quantifies the fraction of the $H$-step-ahead forecast error variance of variable $i$ attributable to shocks in variable $j$. The **Generalized Variance Decomposition (GVD)** method by Koop et al. (1996) and Pesaran and Shin (1998) is used because it is invariant to variable ordering.



 The GVD element is:



 $$

\theta^{g}_{ij}(H) = \frac{\sigma_{jj}^{-1} \sum_{h=0}^{H-1} \left( e_i' A_h \Sigma e_j \right)^2 }{

\sum_{h=0}^{H-1} \left( e_i' A_h \Sigma A_h' e_i \right) }

$$



 where $\sigma_{jj}$ is the variance of the error term for equation $j$, and $e_i$ is a selection vector with 1 in position $i$, zero elsewhere.



 Normalizing rows so their sum equals one gives:



$$

\tilde{\theta}^{g}_{ij}(H) = \frac{\theta^{g}_{ij}(H)}{\sum_{j=1}^N \theta^{g}_{ij}(H)}.

$$



 The **Total Spillover Index (TSI)** measures the contribution of spillovers to total forecast error variance:



$$

S^{g}(H) = \frac{\sum_{i=1}^N \sum_{j=1, j\neq i}^N \tilde{\theta}^{g}_{ij}(H)}{N} \times 100.

$$



Directional spillovers transmitted from market $i$ to others and received by market $i$ from others can also be computed from normalized GVD elements.

In [2]:
# Import libraries
import pandas as pd
import numpy as np
from statsmodels.tsa.api import VAR
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, clear_output

 ## Step 1: Load PIT-transformed realized variances and prepare VAR input

In [3]:
# --- Create dropdown ---
option_selector = widgets.Dropdown(
    options=['europe', 'australia'],
    value='europe',  # initial default
    description='Dataset:',
    disabled=False,
)

# --- Create button ---
button = widgets.Button(
    description="Load Dataset",
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to load selected dataset',
    icon='check'  # (optional) FontAwesome icon
)

# --- Define button click handler ---
def on_button_click(b):
    global selected_option
    clear_output(wait=True)  # Clears previous output to keep it clean
    display(option_selector, button)  # Re-display widgets after clear
    
    selected_option = option_selector.value
    
# --- Attach button click handler ---
button.on_click(on_button_click)

# --- Display UI ---
display(option_selector, button)

Dropdown(description='Dataset:', index=1, options=('europe', 'australia'), value='australia')

Button(button_style='success', description='Load Dataset', icon='check', style=ButtonStyle(), tooltip='Click t…

In [4]:


pit_vars = pd.read_parquet(f"parquet_files/pit_transformed_variances_{selected_option}.parquet")
pit_vars.index = pd.to_datetime(pit_vars.index)

# Inspect the data
pit_vars.head()


Area,nsw,qld,sa,tas,vic
2009-07-01,-0.699212,-1.271926,-0.097437,-0.011362,-0.901846
2009-07-02,0.282052,1.040695,0.046172,-0.024856,0.442966
2009-07-03,-0.661619,-0.563926,-0.589925,-0.683880,-0.319200
2009-07-04,-0.720222,-0.947581,-0.711049,-0.065375,-0.551482
2009-07-05,-0.212486,-0.117435,-0.153986,-0.218300,0.083889


 ## Step 2: Fit VAR($p$) model on realized variances



 Select lag order $p$ (e.g., 1 to 5) based on criteria or domain knowledge.

 Here we use $p=30$.

In [5]:
p = 30
model = VAR(pit_vars)
results = model.fit(p)

print(results.summary())


C:\Users\frolo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Tue, 03, Jun, 2025
Time:                     17:33:27
--------------------------------------------------------------------
No. of Equations:         5.00000    BIC:                   -4.25959
Nobs:                     3500.00    HQIC:                  -5.11423
Log likelihood:          -14296.6    FPE:                 0.00374164
AIC:                     -5.58850    Det(Omega_mle):      0.00302931
--------------------------------------------------------------------
Results for equation nsw
             coefficient       std. error           t-stat            prob
--------------------------------------------------------------------------
const           0.001479         0.012810            0.115           0.908
L1.nsw          0.037553         0.035661            1.053           0.292
L1.qld          0.238104         0.028212            8.440           0.000
L1.sa      

 ## Step 3: Compute moving average coefficient matrices $A_h$



 Calculate matrices $A_h$ for $h=0,\dots,H-1$, using the recursion:



$$

A_0 = I_N, \quad A_h = \sum_{j=1}^p \Phi_j A_{h-j} \quad \text{for } h>0,

$$



 where $\Phi_j$ are the VAR coefficient matrices.

In [6]:
H = 30 

N = pit_vars.shape[1]
I = np.eye(N)
A = [I]

Phi = results.coefs  # shape (p, N, N)

for h in range(1, H):
    A_h = np.zeros((N,N))
    for j in range(1, min(h,p)+1):
        A_h += Phi[j-1] @ A[h-j]
    A.append(A_h)


 ## Step 4: Compute Generalized Variance Decomposition matrix $\Theta^g(H)$

In [7]:
Sigma = results.sigma_u  # covariance matrix of residuals
sigma_diag_inv = np.diag(1 / np.diag(Sigma))

theta_g = np.zeros((N,N))

for i in range(N):
    e_i = np.zeros(N)
    e_i[i] = 1
    denom = 0
    for h in range(H):
        Ah = A[h]
        denom += e_i @ Ah @ Sigma @ Ah.T @ e_i
    for j in range(N):
        e_j = np.zeros(N)
        e_j[j] = 1
        numer = 0
        for h in range(H):
            Ah = A[h]
            val = e_i @ Ah @ Sigma @ e_j
            numer += val ** 2
        theta_g[i,j] = sigma_diag_inv[j,j] * numer / denom


 ## Step 5: Normalize the variance decomposition matrix row-wise

In [8]:
theta_g_normalized = theta_g / theta_g.sum(axis=1, keepdims=True)


 ## Step 6: Compute Total Spillover Index (TSI)

In [9]:
TSI = 100 * (np.sum(theta_g_normalized) - np.trace(theta_g_normalized)) / N
print(f"Total Spillover Index (TSI) at horizon {H}: {TSI:.2f}%")


Total Spillover Index (TSI) at horizon 30: 49.63%


 ## Step 7: Compute directional spillovers transmitted and received by each market



 - Spillovers transmitted from market $i$:



$$

S^{g}_{.i}(H) = \frac{1}{N} \sum_{j=1, j \neq i}^N \tilde{\theta}^{g}_{ji}(H) \times 100

$$



- Spillovers received by market $i$:



$$

S^{g}_{i.}(H) = \frac{1}{N} \sum_{j=1, j \neq i}^N \tilde{\theta}^{g}_{ij}(H) \times 100

$$

In [13]:
# Step 7: Generate Spillover Table 

# --- Core N x N block ---
spillover_matrix = theta_g_normalized * 100  # convert to percent

# --- Directional TO others ---
directional_to = spillover_matrix.sum(axis=1) - np.diag(spillover_matrix)

# --- Directional FROM others ---
directional_from = spillover_matrix.sum(axis=0) - np.diag(spillover_matrix)

# --- NET Directional ---
net_directional = directional_to - directional_from

# --- Create full table ---
spillover_table = pd.DataFrame(spillover_matrix, 
                               index=pit_vars.columns, 
                               columns=pit_vars.columns)

# Add "Directional FROM others" as a column
spillover_table["Directional FROM others"] = directional_from  # REMOVE .values

# Add "Directional TO others" and "NET Directional" as new rows
spillover_table.loc["Directional TO others"] = list(directional_to) + [directional_to.sum()]
spillover_table.loc["NET Directional"] = list(net_directional) + [np.nan]

# --- Display with TSI ---

spillover_table


Area,nsw,qld,sa,tas,vic,Directional FROM others
Area,,,,,,
nsw,37.036545,18.908615,18.102802,10.749657,15.202381,71.015808
qld,24.983939,54.452429,5.406654,4.442690,10.714288,30.509969
sa,19.562020,2.732310,47.855038,5.585195,24.265437,51.291701
tas,12.984357,2.720581,6.244176,63.609044,14.441842,30.686291
vic,13.485493,6.148462,21.538069,9.908749,48.919228,64.623947
Directional TO others,62.963455,45.547571,52.144962,36.390956,51.080772,248.127716
NET Directional,-8.052354,15.037602,0.853261,5.704666,-13.543175,NaN


 ## Save spillover measures

In [14]:
spillover_table.to_parquet(f"parquet_files/volatility_spillovers_{option_selector.value}.parquet")
